<a href="https://colab.research.google.com/github/SurajDonthi/Multi-Camera-Person-Re-Identification/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from argparse import Namespace

from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
# from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers.test_tube import TestTubeLogger

from mtmct_reid.data import ReIDDataModule
from mtmct_reid.engine import ST_ReID
from mtmct_reid.utils import save_args

import warnings

warnings.filterwarnings('ignore')

# Training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

In [ ]:
args = dict(
    description = "Training and Predicting on Market-1501 Dataset",
    data_dir = "data/raw/Market-1501",
    dataset = "market",
    log_path = "logs/",

    learning_rate = 0.01,
    criterion = "cross_entropy",
    rerank = False,
    # random_erasing = True,
    # color_jitter = True,

    gpus = 1,
    max_epochs = 60,
    precision = 16,
    train_batchsize = 64,
    val_batchsize = 64,
    test_batchsize = 64,
    limit_val_batches = 1,
    profiler = True,
    progress_bar_refresh_rate = 20
)

args = Namespace(**args)

In [ ]:
tt_logger = TestTubeLogger(save_dir=args.log_path, name="",
                            description=args.description,
                            )

log_dir = Path(tt_logger.save_dir) / f"version_{tt_logger.version}"

checkpoint_dir = log_dir / "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
chkpt_callback = ModelCheckpoint(checkpoint_dir,
                                    monitor='Loss/val_loss/dataloader_idx_2',   # Pytorch-lightning appends extra path
                                    save_last=True,
                                    mode='min',
                                    save_top_k=10,
                                    )

data_module = ReIDDataModule.from_argparse_args(args)

model = ST_ReID(data_module.num_classes, learning_rate=args.learning_rate,
                criterion=args.criterion, rerank=args.rerank)

save_args(args, log_dir)

trainer = Trainer.from_argparse_args(args, logger=[tt_logger],
                                        checkpoint_callback=chkpt_callback,
                                        profiler=True)  # AdvancedProfiler()

# 
trainer.fit(model, data_module)

# Testing/Evaluation

Use the model to predict matches from other cameras.

In [ ]:
# Testing the loss & accuracy of the model
trainer.test(model)

# Save the results

In [ ]:
from google.colab import files
files.download("/content/experiment_results.zip")